<a href="https://colab.research.google.com/github/Ethereios/Ethereios/blob/main/Copy_of_pyttitools_PYTTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTTI-Tools Colab Notebook

If you are using PyTTI-tools from a local jupyter server, you might have a better experience with the "_local" notebook: https://github.com/pytti-tools/pytti-notebook/blob/main/pyttitools-PYTTI_local.ipynb

If you are planning to use google colab with the "local runtime" option: this is still the notebook you want.

## A very brief history of this notebook

The tools and techniques below were pioneered in 2021 by a diverse and distributed collection of amazingly talented ML practitioners, researchers, and artists. The short version of this history is that Katherine Crowson ([@RiversHaveWings](https://twitter.com/RiversHaveWings)) published a notebook inspired by work done by [@advadnoun](https://twitter.com/advadnoun). Katherine's notebook spawned a litany of variants, each with their own twist on the technique or adding a feature to someone else's work. Henry Rachootin ([@sportsracer48](https://twitter.com/sportsracer48)) collected several of the most interesting notebooks and stuck the important bits together with bublegum and scotch tape. Thus was born PyTTI, and there was much rejoicing in sportsracer48's patreon, where it was shared in closed beta for several months. David Marx ([@DigThatData](https://twitter.com/DigThatData)) offered to help tidy up the mess, and sportsracer48 encouraged him to run wild with it. David's contributions snowballed into [PyTTI-Tools](https://github.com/pytti-tools), the engine this notebook sits on top of!

If you would like to contribute, receive support, or even just suggest an improvement to the documentation, our issue tracker can be found here: https://github.com/pytti-tools/pytti-core/issues

# Instructions

Detailed documentation can be found here: https://pytti-tools.github.io/pytti-book/intro.html

* Syntax for text prompts and scenes: https://pytti-tools.github.io/pytti-book/SceneDSL.html
* Descriptions of all settings: https://pytti-tools.github.io/pytti-book/Settings.html


# Step 1. Setup the environment

In [ ]:
# @title 1.1 Set up storage locations { display-mode: "form" }

drive_mounted = False
gdrive_fpath = '.'

#@markdown Mounting your google drive is optional but recommended. You can even restore from google randomly
#@markdown kicking you out if you mount your drive.

from pathlib import Path

mount_gdrive = True # @param{type:"boolean"}

if mount_gdrive and not drive_mounted:
  from google.colab import drive

  gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
  gdrive_subdirectory = 'MyDrive/pytti_tools' #@param{type:"string"}
  gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
  try:
    drive.mount(gdrive_mountpoint, force_remount = True)
    !mkdir -p {gdrive_fpath}
    %cd {gdrive_fpath}
    drive_mounted = True
  except OSError:
    print(
        "\n\n-----[PYTTI-TOOLS]-------\n\n"
        "If you received a scary OSError and your drive"
        " was already mounted, ignore it."
        "\n\n-----[PYTTI-TOOLS]-------\n\n"
        )
    raise

Mounted at /content/drive/
/content/drive/MyDrive/pytti_tools


In [ ]:
# @title 1.2 Check GPU { display-mode: "form"}

# @markdown Running this cell just gives you information about the GPU attached to your session.

#https://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf
#!nvidia-smi --query-gpu=timestamp,name,utilization.gpu,utilization.memory,memory.free,memory.used --format=csv 

import pandas as pd
import subprocess

outv = subprocess.run(['nvidia-smi', '--query-gpu=timestamp,name,utilization.gpu,utilization.memory,memory.free,memory.used', '--format=csv'], stdout=subprocess.PIPE).stdout.decode('utf-8')

header, rec = outv.split('\n')[:-1]
pd.DataFrame({k:v for k,v in zip(header.split(','), rec.split(','))}, index=[0]).T

,0
timestamp,2022/08/02 02:08:23.265
name,Tesla V100-SXM2-16GB
utilization.gpu [%],0 %
utilization.memory [%],0 %
memory.free [MiB],16160 MiB
memory.used [MiB],0 MiB


In [ ]:
%%capture
#@title 1.3 Install everything else
#@markdown Run this cell on a fresh runtime to install the libraries and modules.

#@markdown This may take a few minutes. 

from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}

def install_pip_deps():
    !pip install kornia pytorch-lightning transformers
    !pip install jupyter loguru einops PyGLM ftfy regex tqdm hydra-core exrex
    !pip install seaborn adjustText bunch matplotlib-label-lines
    !pip install --upgrade gdown

def instal_gh_deps():
  # not sure the "upgrade" arg does anything here, just feels like a good idea
  !pip install --upgrade git+https://github.com/pytti-tools/AdaBins.git
  !pip install --upgrade git+https://github.com/pytti-tools/GMA.git
  !pip install --upgrade git+https://github.com/pytti-tools/taming-transformers.git
  !pip install --upgrade git+https://github.com/openai/CLIP.git
  !pip install --upgrade git+https://github.com/pytti-tools/pytti-core.git

try:
    import pytti
except:
    install_pip_deps()
    instal_gh_deps()

# Preload unopinionated defaults
# makes it so users don't have to run every setup cell
from omegaconf import OmegaConf

!python -m pytti.warmup

path_to_default = 'config/default.yaml'
params = OmegaConf.load(path_to_default)


# setup for step 2

import math

model_default = None
random_seed = None
seed = random_seed
all  = math.inf
derive_from_init_aspect_ratio = -1

########################

try:
    import mmc
except:
    # install mmc
    !git clone https://github.com/dmarx/Multi-Modal-Comparators
    !pip install poetry
    !cd Multi-Modal-Comparators; poetry build
    !cd Multi-Modal-Comparators; pip install dist/mmc*.whl
    
    # optional final step:
    #poe napm_installs
    !python Multi-Modal-Comparators/src/mmc/napm_installs/__init__.py
# suppress mmc warmup outputs
import mmc.loaders

# Step 2: Configure Experiment

Edit the parameters, or load saved parameters, then run the model.

* https://pytti-tools.github.io/pytti-book/SceneDSL.html
* https://pytti-tools.github.io/pytti-book/Settings.html

To input previously used settings or settings generated using tools such as https://pyttipanna.xyz/ , jump down to cell 4.1

In [ ]:
# @title Prompt Settings { display-mode: 'form' } 

scenes = "fractal crystal growth | electric pulse | psychedelic | cyberpunk city || cybernetic ecosystem | microscopic systems | molecular machines | futuristic architecture" # @param{type:"string"}
scene_suffix = " watermark:-1:-.95 | text:-2:-1.95 | " # @param{type:"string"}
scene_prefix = "fractals #unrealengine |  photorealism | psychedelic |" # @param{type:"string"}

params.scenes = scenes
params.scene_prefix = scene_prefix 
params.scene_suffix = scene_suffix


direct_image_prompts   = "https://scontent.fbom12-1.fna.fbcdn.net/v/t39.30808-6/296635124_5552201578180563_3882948280383018360_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=0debeb&_nc_ohc=mQciIZD3zCMAX90q9XT&tn=KGbVnJrgADu2FtZM&_nc_ht=scontent.fbom12-1.fna&oh=00_AT9OKPwq7Wi8aZffh-Hvr0VPVjNRUtrg5DV5E7JFcznQ4g&oe=62E8EEFB" # @param{type:"string"}
init_image = "https://scontent.fbom12-1.fna.fbcdn.net/v/t39.30808-6/296635124_5552201578180563_3882948280383018360_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=0debeb&_nc_ohc=zz5P_5vJ4_0AX91QIoa&tn=KGbVnJrgADu2FtZM&_nc_ht=scontent.fbom12-1.fna&oh=00_AT9xFmVdIF5vmp9VqnfsqaKmKMewcYPYYljcm9uf3om1xQ&oe=62E8EEFB" # @param{type:"string"}
direct_init_weight =  "" # @param{type:"string"}
semantic_init_weight = "" # @param{type:"string"}

params.direct_image_prompts = direct_image_prompts
params.init_image = init_image
params.direct_init_weight = direct_init_weight
params.semantic_init_weight = semantic_init_weight


interpolation_steps = 200 # @param{type:"number"}
steps_per_scene =  60000 # @param{type:"raw"}
steps_per_frame =  150 # @param{type:"number"}
save_every = steps_per_frame  # @param{type:"raw"}

params.interpolation_steps = interpolation_steps
params.steps_per_scene = steps_per_scene
params.steps_per_frame = steps_per_frame
params.save_every = save_every

In [ ]:
# @title Misc Run Initialization { display-mode: 'form' } 

import random

#@markdown Check this box to pick up where you left off from a previous run, e.g. if the google colab runtime timed out
resume = True #@param{type:"boolean"}
params.resume = resume

seed = random_seed #@param{type:"raw"}

params.seed = seed
if params.seed is None:
    params.seed = random.randint(-0x8000_0000_0000_0000, 0xffff_ffff_ffff_ffff)

## Image Settings

In [ ]:
# @title General Image Settings { display-mode: 'form' } 

#@markdown Use `image_model` to select how the model will encode the image
image_model = "Unlimited Palette" #@param ["VQGAN", "Limited Palette", "Unlimited Palette"]
params.image_model = image_model

#@markdown image_model | description | strengths | weaknesses
#@markdown --- | -- | -- | --
#@markdown  VQGAN | classic VQGAN image | smooth images | limited datasets, slow, VRAM intesnsive 
#@markdown  Limited Palette | pytti differentiable palette | fast,  VRAM scales with `palettes` | pixel images
#@markdown  Unlimited Palette | simple RGB optimization | fast, VRAM efficient | pixel images

vqgan_model = "openimages" #@param ["imagenet", "coco", "wikiart", "sflckr", "openimages"]
params.vqgan_model = vqgan_model

#@markdown The output image resolution will be `width` $\times$ `pixel_size` by height $\times$ `pixel_size` pixels.
#@markdown The easiest way to run out of VRAM is to select `image_model` VQGAN without reducing
#@markdown `pixel_size` to $1$.
#@markdown For `animation_mode: 3D` the minimum resoultion is about 450 by 400 pixels.


width =  1280 # @param {type:"raw"}
height =   720# @param {type:"raw"}

params.width = width
params.height = height

#@markdown the default learning rate is `0.1` for all the VQGAN models
#@markdown except openimages, which is `0.15`. For the palette modes the
#@markdown default is `0.02`. 
learning_rate =  model_default #@param{type:"raw"}
reset_lr_each_frame = True #@param{type:"boolean"}

params.learning_rate = learning_rate
params.reset_lr_each_frame = reset_lr_each_frame


In [ ]:
# @title Advanced Color and Appearance options { display-mode: 'form', run: 'auto' } 

pixel_size = 1#@param{type:"number"}
smoothing_weight =  0.2#@param{type:"number"}

params.pixel_size = pixel_size
params.smoothing_weight = smoothing_weight


#@markdown "Limited Palette" specific settings:

random_initial_palette = False#@param{type:"boolean"}
palette_size = 6#@param{type:"number"}
palettes   = 9#@param{type:"number"}

params.random_initial_palette = random_initial_palette
params.palette_size = palette_size
params.palettes = palettes


gamma = 1#@param{type:"number"}
hdr_weight = 0.01#@param{type:"number"}
palette_normalization_weight = 0.2#@param{type:"number"}
target_palette = ""#@param{type:"string"}
lock_palette = False #@param{type:"boolean"}
show_palette = False #@param{type:"boolean"}

params.gamma = gamma
params.hdr_weight = hdr_weight
params.palette_normalization_weight = palette_normalization_weight
params.target_palette = target_palette
params.lock_palette = lock_palette
params.show_palette = show_palette

## Perceptor Settings

In [ ]:
# @title Perceptor Models { display-mode: 'form', run: 'auto' } 

#@markdown Quality settings from Dribnet's CLIPIT (https://github.com/dribnet/clipit).
#@markdown Selecting too many will use up all your VRAM and slow down the model.
#@markdown I usually use ViTB32, ViTB16, and RN50 if I get a A100, otherwise I just use ViT32B.

#@markdown quality | CLIP models
#@markdown --- | --
#@markdown  draft | ViTB32 
#@markdown  normal | ViTB32, ViTB16 
#@markdown  high | ViTB32, ViTB16, RN50
#@markdown  best | ViTB32, ViTB16, RN50x4

# To do: change this to a multi-select

ViTB32 = True #@param{type:"boolean"}
ViTB16 = True #@param{type:"boolean"}
ViTL14  = False #@param{type:"boolean"}
ViTL14_336px  = False #@param{type:"boolean"}
RN50 = True #@param{type:"boolean"}
RN101 = False #@param{type:"boolean"}
RN50x4 = False #@param{type:"boolean"}
RN50x16 = False #@param{type:"boolean"}
RN50x64 = False #@param{type:"boolean"}


params.ViTB32 = ViTB32
params.ViTB16 = ViTB16
params.ViTL14 = ViTL14
params.ViTL14_336px = ViTL14_336px
params.RN50 = RN50
params.RN101 = RN101
params.RN50x4 = RN50x4
params.RN50x16 = RN50x16
params.RN50x64 = RN50x64



In [ ]:
# @title MMC Perceptors { display-mode: 'form' } 

#@markdown This cell loads perceptor models via https://github.com/dmarx/multi-modal-comparators. Some model comparisons [here](https://t.co/iShJpm5GjL)

# @markdown Select up to three models


# @markdown Model 1
model1 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - openai - RN50x4]","[clip - openai - RN50x16]","[clip - openai - RN50x64]","[clip - openai - ViT-B/32]","[clip - openai - ViT-B/16]","[clip - openai - ViT-L/14]","[clip - openai - ViT-L/14@336px]","[clip - mlfoundations - RN50--openai]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--openai]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--openai]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--openai]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - RN50x4--openai]","[clip - mlfoundations - RN50x16--openai]","[clip - mlfoundations - ViT-B-32--openai]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--openai]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--openai]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - mlfoundations - ViT-L-14--openai]","[clip - mlfoundations - ViT-L-14-336--openai]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - sajjjadayobi - clipfa]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_16_epochs]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_32_epochs]","[clip - navervision - kelip_ViT-B/32]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model2 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - openai - RN50x4]","[clip - openai - RN50x16]","[clip - openai - RN50x64]","[clip - openai - ViT-B/32]","[clip - openai - ViT-B/16]","[clip - openai - ViT-L/14]","[clip - openai - ViT-L/14@336px]","[clip - mlfoundations - RN50--openai]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--openai]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--openai]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--openai]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - RN50x4--openai]","[clip - mlfoundations - RN50x16--openai]","[clip - mlfoundations - ViT-B-32--openai]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--openai]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--openai]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - mlfoundations - ViT-L-14--openai]","[clip - mlfoundations - ViT-L-14-336--openai]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - sajjjadayobi - clipfa]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_16_epochs]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_32_epochs]","[clip - navervision - kelip_ViT-B/32]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model3 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - openai - RN50x4]","[clip - openai - RN50x16]","[clip - openai - RN50x64]","[clip - openai - ViT-B/32]","[clip - openai - ViT-B/16]","[clip - openai - ViT-L/14]","[clip - openai - ViT-L/14@336px]","[clip - mlfoundations - RN50--openai]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--openai]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--openai]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--openai]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - RN50x4--openai]","[clip - mlfoundations - RN50x16--openai]","[clip - mlfoundations - ViT-B-32--openai]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--openai]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--openai]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - mlfoundations - ViT-L-14--openai]","[clip - mlfoundations - ViT-L-14-336--openai]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - sajjjadayobi - clipfa]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_16_epochs]","[cloob - crowsonkb - cloob_laion_400m_vit_b_16_32_epochs]","[clip - navervision - kelip_ViT-B/32]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}

##########

params.use_mmc = False
mmc_models = []

for model_key in (model1, model2, model3):
    if not model_key:
        continue
    arch, pub, m_id = model_key[1:-1].split(' - ')
    params.use_mmc = True
    mmc_models.append({
        'architecture':arch,
        'publisher':pub,
        'id':m_id,
        })
params.mmc_models = mmc_models 


In [ ]:
# @title Cutouts { display-mode: 'form', run: 'auto' } 

#@markdown [Cutouts are how CLIP sees the image.](https://twitter.com/remi_durant/status/1460607677801897990)

cutouts = 40#@param{type:"number"}
cut_pow = 2#@param {type:"number"}
cutout_border =  .25#@param {type:"number"}
gradient_accumulation_steps = 1 #@param {type:"number"}


params.cutouts = cutouts
params.cut_pow = cut_pow
params.cutout_border = cutout_border
params.gradient_accumulation_steps = gradient_accumulation_steps

## Animation Settings

In [ ]:
# @title General Animation Settings { display-mode: 'form', run: 'auto' } 

animation_mode = "3D" #@param ["off","2D", "3D", "Video Source"]
pre_animation_steps =  250 # @param{type:"number"}
frames_per_second =  40 # @param{type:"number"}

params.animation_mode = animation_mode
params.pre_animation_steps = pre_animation_steps
params.frames_per_second = frames_per_second


# @markdown NOTE: prompt masks (`prompt:weight_[mask.png]`) may not work correctly on '`wrap`' or '`mirror`' border mode.
border_mode = "clamp" # @param ["clamp","mirror","wrap","black","smear"]
sampling_mode = "bicubic" #@param ["bilinear","nearest","bicubic"]
infill_mode = "wrap" #@param ["mirror","wrap","black","smear"]

params.border_mode = border_mode
params.sampling_mode = sampling_mode
params.infill_mode = infill_mode

In [ ]:
# @title Video Input { display-mode: 'form', run: 'auto' } 

video_path = ""# @param{type:"string"}
frame_stride = 1 #@param{type:"number"}
reencode_each_frame = False #@param{type:"boolean"}

params.video_path = video_path
params.frame_stride = frame_stride
params.reencode_each_frame = reencode_each_frame

In [ ]:
# @title Audio Input { display-mode: 'form', run: 'auto' } 

input_audio = "/content/drive/MyDrive/compositions/ethereios sleepless monk/mystic dance/5 Ethereios vs James - Mystic Flow - Master 24 Bit.wav"# @param{type:"string"}
input_audio_offset = 0 #@param{type:"number"}

# @markdown Bandpass filter specification

variable_name = 'fAudio'
f_center = 1000 # @param{type:"number"}
f_width = 1990 # @param{type:"number"}
order = 5 # @param{type:"number"}

if input_audio:
  params.input_audio = input_audio
  params.input_audio_offset = input_audio_offset
  params.input_audio_filters = [{
      'variable_name':variable_name,
      'f_center':f_center,
      'f_width':f_width,
      'order':order
    }]


In [ ]:
# @title Image Motion Settings  { display-mode: 'form', run: 'auto' } 

# @markdown settings whose names end in `_2d` or `_3d` are specific to those animation modes

# @markdown `rotate_3d` *must* be a `[w,x,y,z]` rotation (unit) quaternion. Use `rotate_3d: [1,0,0,0]` for no rotation.

# @markdown [Learn more about rotation quaternions here](https://eater.net/quaternions).

translate_x = "-1700*sin(radians(1.5))" # @param{type:"string"}
translate_y = "0" # @param{type:"string"}
translate_z_3d = "666*sin((6.66+t)/3)**16" # @param{type:"string"}
rotate_3d = "[cos(radians(1.5)), 0, -sin(radians(1.5))/sqrt(2), sin(radians(1.5))/sqrt(2)]" # @param{type:"string"}
rotate_2d = "0.03" # @param{type:"string"}
zoom_x_2d = "0.01" # @param{type:"string"}
zoom_y_2d = "0.01" # @param{type:"string"}

params.translate_x = translate_x
params.translate_y = translate_y
params.translate_z_3d = translate_z_3d
params.rotate_3d = rotate_3d
params.rotate_2d = rotate_2d
params.zoom_x_2d = zoom_x_2d
params.zoom_y_2d = zoom_y_2d



#@markdown  3D camera (only used in 3D mode):
lock_camera   = True # @param{type:"boolean"}
field_of_view = 120 # @param{type:"number"}
near_plane    = 1 # @param{type:"number"}
far_plane     = 10000 # @param{type:"number"}

params.lock_camera = lock_camera
params.field_of_view = field_of_view
params.near_plane = near_plane
params.far_plane = far_plane

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Stabilization Weights and Perspective { display-mode: 'form', run: 'auto' } 

# @markdown `flow_stabilization_weight` is used for `animation_mode: 3D` and `Video Source`

direct_stabilization_weight = "0.1" # @param{type:"string"}
semantic_stabilization_weight = "0.1" # @param{type:"string"}
depth_stabilization_weight = "0.05" # @param{type:"string"}
edge_stabilization_weight = "0.1" # @param{type:"string"}

params.direct_stabilization_weight = direct_stabilization_weight
params.semantic_stabilization_weight = semantic_stabilization_weight
params.depth_stabilization_weight = depth_stabilization_weight
params.edge_stabilization_weight = edge_stabilization_weight


flow_stabilization_weight = "0.1" # @param{type:"string"}
flow_long_term_samples = 1 # @param{type:"number"}

params.flow_stabilization_weight = flow_stabilization_weight
params.flow_long_term_samples = flow_long_term_samples



## Output Settings

In [ ]:
# @title Output and Storage Location { display-mode: 'form', run: 'auto' } 

# should I move google drive stuff here?

models_parent_dir = '.' #@param{type:"string"}
params.models_parent_dir = models_parent_dir

file_namespace = "twistedukiyo" #@param{type:"string"}
params.file_namespace = file_namespace
if params.file_namespace == '':
  params.file_namespace = 'out'


allow_overwrite = False #@param{type:"boolean"}
base_name = params.file_namespace

params.allow_overwrite = allow_overwrite
params.base_name = base_name


#@markdown `backups` is used for video transfer, so don't lower it if that's what you're doing
backups =  2**(params.flow_long_term_samples+1)+1 #@param {type:"raw"}
params.backups = backups

from pytti.Notebook import get_last_file

import glob
import re
# to do: move this logic into pytti-core
if not params.allow_overwrite and path_exists(f'images_out/{params.file_namespace}'):
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                        f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  if i == 0:
    print(f"WARNING: file_namespace {params.file_namespace} already has images from run 0")
  elif i is not None:
    print(f"WARNING: file_namespace {params.file_namespace} already has images from runs 0 through {i}")
elif glob.glob(f'images_out/{params.file_namespace}/{params.base_name}_*.png'):
  print(f"WARNING: file_namespace {params.file_namespace} has images which will be overwritten")

In [ ]:
# @title Experiment Monitoring { display-mode: 'form', run: 'auto' } 

display_every = steps_per_frame # @param{type:"raw"}
clear_every = 0 # @param{type:"raw"}
display_scale = 1 # @param{type:"number"}

params.display_every = display_every
params.clear_every = clear_every
params.display_scale = display_scale

show_graphs = True # @param{type:"boolean"}
use_tensorboard = True #@param{type:"boolean"}

params.show_graphs = show_graphs
params.use_tensorboard = use_tensorboard

# needs to be populated or will fail validation
params.approximate_vram_usage=False



In [ ]:
print("SETTINGS:")
print(OmegaConf.to_container(params))

# 2.3 Run it!

In [ ]:
#@markdown Execute this cell to start image generation
from pytti.workhorse import _main as render_frames
import random

if (seed is None) or (params.seed is None):
  params.seed = random.randint(-0x8000_0000_0000_0000, 0xffff_ffff_ffff_ffff)

render_frames(params)

# Step 3: Render video
You can dowload from the notebook, but it's faster to download from your drive.

In [ ]:
#@title 3.1 Render video
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import change_tqdm_color
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()
  
from tqdm.notebook import tqdm
import numpy as np
from os.path import exists as path_exists
from subprocess import Popen, PIPE
from PIL import Image, ImageFile
from os.path import splitext as split_file
import glob
from pytti.Notebook import get_last_file

ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
  params
except NameError:
  raise RuntimeError("ERROR: no parameters. Please run parameters (step 2.1).")

if not path_exists(f"images_out/{params.file_namespace}"):
  if path_exists(f"/content/drive/MyDrive"):
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"ERROR: file_namespace: {params.file_namespace} does not exist.")
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: file_namespace: {params.file_namespace} does not exist.")

#@markdown The first run executed in `file_namespace` is number $0$, the second is number $1$, etc.

latest = -1
run_number = latest#@param{type:"raw"}
if run_number == -1:
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  run_number = i
base_name = params.file_namespace if run_number == 0 else (params.file_namespace+f"({run_number})")
tqdm.write(f'Generating video from {params.file_namespace}/{base_name}_*.png')

all_frames = glob.glob(f'images_out/{params.file_namespace}/{base_name}_*.png')
all_frames.sort(key = lambda s: int(split_file(s)[0].split('_')[-1]))
print(f'found {len(all_frames)} frames matching images_out/{params.file_namespace}/{base_name}_*.png')

start_frame = 0#@param{type:"number"}
all_frames = all_frames[start_frame:]

fps =  params.frames_per_second#@param{type:"raw"}

total_frames = len(all_frames)

if total_frames == 0:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError(f"ERROR: no frames to render in images_out/{params.file_namespace}")

frames = []

for filename in tqdm(all_frames):
  frames.append(Image.open(filename))

cmd_in = ['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-']
cmd_out = ['-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f'videos/{base_name}.mp4']
if params.input_audio:
  cmd_in += ['-i', str(params.input_audio), '-acodec', 'libmp3lame']

cmd = cmd_in + cmd_out

p = Popen(cmd, stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video complete.")

In [ ]:
#@title 3.2 Download the last exported video
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}

try:
  from pytti.Notebook import get_last_file
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')

try:
  params
except NameError:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError("ERROR: please run parameters (step 2.1).")

from google.colab import files
try:
  base_name = params.file_namespace if run_number == 0 else (params.file_namespace+f"({run_number})")
  filename = f'{base_name}.mp4'
except NameError:
  filename, i = get_last_file(f'videos', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?\\.mp4)$')

if path_exists(f'videos/{filename}'):
  files.download(f"videos/{filename}")
else:
  if path_exists(f"/content/drive/MyDrive"):
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"ERROR: video videos/{filename} does not exist.")
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: video videos/{filename} does not exist.")

# Sec. 4: Appendix

In [ ]:
#@title 4.1 Load settings (optional)
#@markdown copy the `SETTINGS:` output from the **Parameters** cell (tripple click to select the whole
#@markdown line from `{'scenes'...` to `}`) and paste them in a note to save them for later.

#@markdown Paste them here in the future to load those settings again. Running this cell with blank settings won't do anything.
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import *
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()
  
import json, random
try:
  from bunch import Bunch
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')

settings = ""#@param{type:"string"}
#@markdown Check `random_seed` to overwrite the seed from the settings with a random one for some variation.
random_seed = False #@param{type:"boolean"}

if settings != '':
  params = load_settings(settings, random_seed)

## 4.2 License

```
Licensed under the MIT License
Copyleft (c) 2021 Henry Rachootin
Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
```